<a href="https://colab.research.google.com/github/plthiyagu/Personnel/blob/master/WebScraping_Amazon_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
!pip3 install beautifulsoup4

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [0]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [0]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [124]:
df.head()

,Book Name,Author,Rating,Customers_Rated,Price
0,The Power of your Subconscious Mind,Joseph Murphy,4.5 out of 5 stars,"13,961",0
1,Think and Grow Rich,Napoleon Hill,4.5 out of 5 stars,"16,704",0
2,Word Power Made Easy,Norman Lewis,4.4 out of 5 stars,"10,721",0
3,The Girl in Room 105,Chetan Bhagat,4.3 out of 5 stars,"5,177",0
4,General Knowledge,Dr. Binay Karna,4.4 out of 5 stars,"5,123",0


In [125]:
df.shape

(100, 5)

In [0]:
df['Rating']= df['Rating'].apply(lambda x: x.split()[0])

In [0]:
df['Rating']=pd.to_numeric(df['Rating'])

In [0]:
#df["Price"] = df["Price"].str.replace('₹', '')

In [0]:
#df["Price"] = df["Price"].str.replace(',', '')

In [0]:
#df['Price'] = df['Price'].apply(lambda x: x.split('.')[0])


In [158]:
df['Price'] = df['Price'].astype(int)

ValueError: ignored

In [0]:
df['Customers_Rated']= df['Customers_Rated'].str.replace(',','')

In [0]:
df['Customers_Rated'] = pd.to_numeric(df['Customers_Rated'], errors='ignore')

In [133]:
df.head()

,Book Name,Author,Rating,Customers_Rated,Price
0,The Power of your Subconscious Mind,Joseph Murphy,4.5,13961,0
1,Think and Grow Rich,Napoleon Hill,4.5,16704,0
2,Word Power Made Easy,Norman Lewis,4.4,10721,0
3,The Girl in Room 105,Chetan Bhagat,4.3,5177,0
4,General Knowledge,Dr. Binay Karna,4.4,5123,0


In [157]:
df.dtypes

Book Name           object
Author              object
Rating             float64
Customers_Rated      int64
Price              float64
dtype: object

In [0]:
df.replace(str(0), np.nan, inplace=True)
df.replace(0, np.nan, inplace=True)

In [0]:
count_nan = len(df) - df.count()

In [137]:
count_nan

Book Name            0
Author               0
Rating               0
Customers_Rated      0
Price              100
dtype: int64

In [0]:
#df = df.dropna()

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Book Name        100 non-null    object 
 1   Author           100 non-null    object 
 2   Rating           100 non-null    float64
 3   Customers_Rated  100 non-null    int64  
 4   Price            0 non-null      float64
dtypes: float64(2), int64(1), object(2)
memory usage: 4.0+ KB


In [0]:
data = df.sort_values(["Price"], axis=0, ascending=False)[:15]

In [141]:
data

,Book Name,Author,Rating,Customers_Rated,Price
0,The Power of your Subconscious Mind,Joseph Murphy,4.5,13961,NaN
1,Think and Grow Rich,Napoleon Hill,4.5,16704,NaN
2,Word Power Made Easy,Norman Lewis,4.4,10721,NaN
3,The Girl in Room 105,Chetan Bhagat,4.3,5177,NaN
4,General Knowledge,Dr. Binay Karna,4.4,5123,NaN
5,Quantitative Aptitude for Competitive Examinat...,R S Aggarwal,4.4,4555,NaN
6,The Subtle Art of Not Giving a F*ck,Mark Manson,4.4,14493,NaN
7,Indian Polity 5th Edition,M. Laxmikanth,4.5,5340,NaN
8,The Alchemist,Paulo Coelho,4.7,22230,NaN
9,How to Win Friends and Influence People,Dale Carnegie,4.6,15417,NaN


In [0]:
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
import math
from bokeh.io import curdoc
curdoc().clear()
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import Legend
output_notebook()

In [0]:
p = figure(x_range=data.iloc[:,1], plot_width=800, plot_height=550, title="Authors Highest Priced Book", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,1], top=data.iloc[:,4], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

In [144]:
show(p)

ERROR:bokeh.core.validation.check:E-1019 (DUPLICATE_FACTORS): FactorRange must specicy a unique list of categorical factors for an axis: duplicate factors found: 'Robin Sharma'


In [0]:
data = df[df['Customers_Rated'] > 1000]

In [0]:
data = data.sort_values(['Rating'],axis=0, ascending=False)[:15]

In [147]:
data

,Book Name,Author,Rating,Customers_Rated,Price
65,Bhagavad Gita: Yatharoop (Hindi),A.C. Bhaktivendanta Swami Prabhupada,4.8,2533,NaN
94,Diary of a Wimpy Kid: Wrecking Ball (Book 14) ...,Jeff Kinney,4.7,6981,NaN
83,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.7,5258,NaN
16,Inner Engineering: A Yogi’s Guide to Joy,Sadhguru,4.7,4105,NaN
67,The Secret Of The Nagas (Shiva Trilogy-2),Amish Tripathi,4.7,2905,NaN
8,The Alchemist,Paulo Coelho,4.7,22230,NaN
37,Harry Potter and the Philosopher's Stone,J.K. Rowling,4.7,7768,NaN
45,Concept of Physics Part-1 (2019-2020 Session) ...,H.C. Verma,4.6,2335,NaN
46,The Rudest Book Ever,Shwetabh Gangwar,4.6,1181,NaN
30,The Theory of Everything,Stephen Hawking,4.6,2004,NaN


In [0]:
p = figure(x_range=data.iloc[:,0], plot_width=800, plot_height=600, title="Top Rated Books with more than 1000 Customers Rating", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,0], top=data.iloc[:,2], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

In [149]:
show(p)

In [0]:
p = figure(x_range=data.iloc[:,1], plot_width=800, plot_height=600, title="Top Rated Books with more than 1000 Customers Rating", toolbar_location=None, tools="")

p.vbar(x=data.iloc[:,1], top=data.iloc[:,2], width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.major_label_orientation = math.pi/2

In [151]:
show(p)

ERROR:bokeh.core.validation.check:E-1019 (DUPLICATE_FACTORS): FactorRange must specicy a unique list of categorical factors for an axis: duplicate factors found: 'J.K. Rowling', 'H.C. Verma'


In [0]:
data = df.sort_values(["Customers_Rated"], axis=0, ascending=False)[:20]

In [0]:
from bokeh.transform import factor_cmap
from bokeh.models import Legend
from bokeh.palettes import Dark2_5 as palette
import itertools
from bokeh.palettes import d3
#colors has a list of colors which can be used in plots
colors = itertools.cycle(palette)

palette = d3['Category20'][20]

In [0]:
index_cmap = factor_cmap('Author', palette=palette,
                         factors=data["Author"])

In [155]:
p = figure(plot_width=700, plot_height=700, title = "Top Authors: Rating vs. Customers Rated")
p.scatter('Rating','Customers_Rated',source=data,fill_alpha=0.6, fill_color=index_cmap,size=20,legend='Author')
p.xaxis.axis_label = 'RATING'
p.yaxis.axis_label = 'CUSTOMERS RATED'
p.legend.location = 'top_left'

In [156]:
show(p)